## This notebook demonstrates the use of a simple meal planner expert system using Canada's Food and Nutrition database as the Knowledge Base

In [13]:
#import libraries
import numpy as np
import pandas as pd
import random
from IPython.display import display

In [23]:
class UserData():
    """This class stores and calculates user specific data"""
    def __init__(self):
        """Initialize User Data Variables"""
        self.age = None
        self.height = None
        self.gender = None
        self.weight = None
        self.prot_req = None
        self.carb_req = None
        self.fat_req = None
        #self.ingredient_df = None
        
    def input_user_data(self,input_list):
        """Use input list to initialize user data"""
        self.age = input_list[0]
        self.height = input_list[1]
        self.gender = input_list[2]
        self.weight = input_list[3]                
        
    def calculate_daily_food_requirements(self):
        """This function uses the Harris-Benedict Equation for Basal Energy Expenditure to 
           to calculate the daily requirement of protein, fat and carbohydrates"""
        if self.gender == 1:
            multipliers = [655.1,9.6,1.9,4.7]
        else:
            multipliers = [66.5,13.8,5.0,6.8]
        #HBE to find BEE
        total_calories = multipliers[0] + multipliers[1] * self.weight\
                        + multipliers[2] * self.height + multipliers[3] * self.age
        self.prot_req = self.weight
        prot_per = ((self.weight * 4)/total_calories) * 100
        carb_per = 60
        fat_per = 100 - (prot_per + carb_per)
        self.carb_req = ((total_calories * carb_per)/100)/4
        self.fat_req = ((total_calories * fat_per)/100)/9
        print("You require at most",self.prot_req,"g of protein",self.carb_req,"g of carbohydrates",\
             self.fat_req,"g of fats per day")

In [24]:
user = UserData()
user.input_user_data([25,178,0,95]) #[Age,Height,Gender(0 for male 1 for female),weight]
user.calculate_daily_food_requirements()

You require at most 95 g of protein 365.625 g of carbohydrates 66.11111111111111 g of fats per day


In [25]:
class Meal():
    """Meal class holds the information for one day's meal"""
    
    def __init__(self,ingredients,user_data):
        """Creates a Random Meal from User Data OR from specified dataframe"""
        self.define_new_meal(ingredients,user_data)
        
    def define_new_meal(self,ingredients,user_data):
        self.fitness = 0.0
        self.ingredients = ingredients
        self.ingredients['MealAmount'] = 0.0
        req_list = [user_data.prot_req + 10,user_data.fat_req + 10, user_data.carb_req + 10]
        for i,row in self.ingredients.iterrows():
            value = self.get_meal_amount(req_list,row)
            self.ingredients.loc[i,'MealAmount'] = value
            
        
    def get_meal_amount(self,req_list,row):
        if row.PROTValue > row.CARBValue and row.PROTValue > row.FATValue: #main source of protein
            value = random.random() * req_list[0]
            req_list[0] = req_list[0] - value
            value = (value * 100)/row.PROTValue
            return value
        if row.FATValue > row.PROTValue and row.FATValue > row.CARBValue:
            value = random.random() * req_list[1]
            req_list[1] = req_list[1] - value
            value = (value * 100)/row.FATValue
            return value
        if row.CARBValue > row.PROTValue and row.CARBValue > row.FATValue:
            value = random.random() * req_list[2]
            req_list[2] = req_list[2] - value
            value = (value * 100)/row.CARBValue
            return value
        return random.random() * 500
        
    def adjust_amount(self,user_data):
        self.ingredients['TotalProt'] = (self.ingredients['PROTValue'] * self.ingredients['MealAmount'])/100
        self.ingredients['TotalCarb'] = (self.ingredients['CARBValue'] * self.ingredients['MealAmount'])/100
        self.ingredients['TotalFat'] = (self.ingredients['FATValue'] * self.ingredients['MealAmount'])/100
        max_prot_id = self.ingredients.TotalProt.idxmax()
        max_carb_id = self.ingredients.TotalCarb.idxmax()
        total_vals = self.get_total_vals()
        prot_diff = ((user_data.prot_req - total_vals['PROTValue']) * 100)/self.ingredients.at[max_prot_id,'PROTValue']
        self.ingredients.loc[max_prot_id,'MealAmount'] = self.ingredients.at[max_prot_id,'MealAmount'] + prot_diff
        carb_diff = ((user_data.carb_req - total_vals['CARBValue']) * 100)/self.ingredients.at[max_carb_id,'CARBValue']
        self.ingredients.loc[max_carb_id,'MealAmount'] = self.ingredients.at[max_carb_id,'MealAmount'] + carb_diff
            
        self.ingredients.drop(columns=['TotalFat','TotalProt','TotalCarb'],inplace=True)
        self.update_fitness(user_data.prot_req,user_data.carb_req,user_data.fat_req)
            
    def display_meal(self):
        display(self.ingredients)
        print(self.get_total_vals())
        
    def get_total_vals(self):
        total_vals = {
            "PROTValue":0.0,
            "FATValue":0.0,
            "CARBValue":0.0,
            "STARValue":0.0,
            "TSUGValue":0.0,
            "TDFValue":0.0,
            "TSATValue":0.0,
            "MUFAValue":0.0,
            "PUFAValue":0.0
        }
        for i,row in self.ingredients.iterrows():
            for column in total_vals:
                per100g = row[column]/100.0
                val = row.MealAmount
                total_vals[column] = total_vals[column] + val * per100g
        return total_vals
               
    def update_fitness(self,prot_req,carb_req,fat_req):
        total_vals = self.get_total_vals()
        #apply rules to calculate fitness
        ing_per = (prot_req - total_vals["PROTValue"])/prot_req
        self.fitness = 0
        if ing_per > 0.7:
            self.fitness = self.fitness + 1
        else:
            self.fitness = self.fitness - 1
        ing_per = (carb_req - total_vals["CARBValue"])/carb_req
        if ing_per > 0.7:
            self.fitness = self.fitness + 1
        else:
            self.fitness = self.fitness - 1
        ing_per = (fat_req - total_vals["FATValue"])/fat_req
        if ing_per > 0.7:
            self.fitness = self.fitness + 1
        else:
            self.fitness = self.fitness - 1
        ing_per = total_vals["TSUGValue"]/total_vals["CARBValue"]
        if ing_per < 0.3: #if sugar is less than 30% of carb value then meal is good
            self.fitness = self.fitness + 1
        else:
            self.fitness = self.fitness - 1
        ing_per = total_vals["TSATValue"]/total_vals["FATValue"]
        if ing_per < 0.3: #if saturated fat is less than 30% of fat value then meal is good
            self.fitness = self.fitness + 1
        else:
            self.fitness = self.fitness - 1
        

class DailyMealPlanner():
    """Implementation of Expert System that gives a new meal plan daily"""
    def __init__(self,
                 user_data,
                 combinations = 100,
                 no_of_ingredients = 5,
                 mutation_rate=0.001,
                 elite_percentage=10,
                 random_state=1):
        """Initialize The Meal Plan System"""
        self.combinations = combinations
        self.no_of_ingredients = no_of_ingredients
        self.mutation_rate = mutation_rate
        self.elite_per_unit = elite_percentage/100.0
        self.random_state = random_state
        self.user_data = user_data
        self.food_data = pd.read_csv('FoodNutritionData.csv')
        self.food_data = self.food_data[~self.food_data.FoodGroup.isin(
            ["Babyfoods","Beverages","Fats and Oils","Spices and Herbs"])].copy()
        self.increment = 0
    
    def create_population(self):
        """Get Random Samples from ingredient data"""
        random.seed(self.random_state)
        self.increment = self.increment + 1
        self.current_generation = [] #create random new meals from user ingredients
        for i in range(0,self.combinations):
            self.current_generation.append(Meal(self.food_data.sample(
                n = self.no_of_ingredients,
                random_state = self.random_state + self.increment),
                                               self.user_data))
            self.increment = self.increment + 1
            
    def update_generation_order(self):
        """Updates the fitness function of the meal and sets the order of the current generation"""
        for meal in self.current_generation:
            meal.update_fitness(self.user_data.prot_req,self.user_data.carb_req,self.user_data.fat_req)
        #rank meals in order of fitness
        self.current_generation = sorted(self.current_generation, key=lambda meal: meal.fitness, reverse=True)
        
    def perform_selection(self):
        """Performs the selection step of genetic algorithm"""
        total_selected = int(self.elite_per_unit * len(self.current_generation))
        selected_generations = []
        for i in range(0,total_selected):
            selected_generations.append(self.current_generation[i])
        # randomly pick candidates from the rest
        selected_generations.extend(random.sample(self.current_generation[int(self.elite_per_unit):], 10))
        self.current_generation = selected_generations
    
    def perform_ordered_cross_over(self):
        """Performs the ordered cross over step of genetic algorithm"""
        #shuffle selected generation
        self.current_generation = random.sample(self.current_generation,len(self.current_generation))
        children = []
        while len(children) < self.combinations:
            x,y = random.sample(range(len(self.current_generation)), 2)
            parent_meal_1 = self.current_generation[x]
            parent_meal_2 = self.current_generation[y]
            p1, p2 = random.sample(range(self.no_of_ingredients), k=2) #get two points in list
            from_index = min(p1, p2) #arrange in min max
            to_index = max(p1, p2)
            parent1_ingredients = parent_meal_1.ingredients.copy() #copy to ensure parent stays same
            parent1_ingredients.drop(columns=['MealAmount'],inplace=True) #drop meal amount since it will be calculated
            child_ingredients = parent1_ingredients.to_dict(orient='records')[from_index:to_index]
            child_food_list = [ingredient['FoodDescription'] for ingredient in child_ingredients]
            parent2_ingredients = parent_meal_2.ingredients.copy()
            parent2_ingredients.drop(columns=['MealAmount'],inplace=True)
            parent_2_genes = parent2_ingredients.to_dict(orient='records')
            for i in range(0,self.no_of_ingredients - len(child_ingredients)): #fill rest with parent 2 records
                ingredient_val = None
                for ingredient in parent_2_genes:
                    if ingredient['FoodDescription'] not in child_food_list:
                        ingredient_val = ingredient
                        break
                if ingredient_val is None: #didn't find unique ingredient, get from dataset
                    ingredient_val = self.get_unique_ingredient(child_food_list)
                child_ingredients.append(ingredient_val)
                child_food_list.append(ingredient_val["FoodDescription"])
            #create child from parent ingredients
            child = Meal(pd.DataFrame(child_ingredients),self.user_data)
            #add to next generation 
            children.append(child)
        self.current_generation = children
        
    def get_unique_ingredient(self,ing_list):
        ingredient_val = None
        while ingredient_val is None:
            ingredient_val = self.food_data.sample(n = 1,random_state = self.random_state + self.increment)\
            .to_dict(orient='records')[0]
            self.increment = self.increment + 1
            if ingredient_val['FoodDescription'] in ing_list:
                ingredient_val = None
        ing_list.append(ingredient_val['FoodDescription'])
        return ingredient_val
        
    def perform_mutation(self):
        """Performs the mutation step of genetic algorithm"""
        for meal in self.current_generation:
            #get ingredients of current meal
            ingredients = meal.ingredients.copy() #copy to ensure nothing changes in first iteration
            ingredients.drop(columns=['MealAmount'], inplace = True) #drop meal amount since it will be recalculated
            ingredients = ingredients.to_dict(orient='records')
            food_list = [ingredient["FoodDescription"] for ingredient in ingredients]
            mutation_count = 0
            for i in range(self.no_of_ingredients):
                if (random.random() < self.mutation_rate):
                    mutation_count = mutation_count + 1
                    food_list.remove(ingredients[i]['FoodDescription'])
                    ingredients[i] = self.get_unique_ingredient(food_list) #replace with fresh ingredient
            meal.define_new_meal(pd.DataFrame(ingredients),self.user_data) #set changes to ingredients dataframe
        
    def next_generation(self):
        """Runs a single iteration of genetic algorithm"""
        self.update_generation_order()
        self.perform_selection()
        self.perform_ordered_cross_over()
        self.perform_mutation()
            
    def get_healthy_meal(self,generations = 100):
        """Runs the Genetic Algorithm to get a new meal"""
        self.create_population()
        for i in range(0,generations): #run for generations
            self.next_generation()
        self.update_generation_order()
        top_meal = self.current_generation[0]
        return top_meal #return best generation

In [26]:
daily_meal_planner = DailyMealPlanner(user,100,10,0.002,10,5)
new_meal = daily_meal_planner.get_healthy_meal(100)

In [27]:
new_meal.adjust_amount(user)
new_meal.display_meal()
print("Meal Fitness: ",new_meal.fitness)

,FoodID,FoodDescription,FoodGroup,PROTValue,FATValue,CARBValue,STARValue,TSUGValue,TDFValue,TSATValue,MUFAValue,PUFAValue,MealAmount
0,3572,"Game meat, bison, roasted","Lamb, Veal and Game",28.44,2.42,0.00,0.00,0.00,0.0,0.910,0.950,0.240,62.458523
1,1505,"Apricot, canned whole no skin, heavy syrup pac...",Fruits and fruit juices,0.51,0.09,21.45,0.00,0.00,1.6,0.006,0.037,0.017,527.824184
2,5185,"Turkey, tom, wing, meat, roasted",Poultry Products,31.81,4.57,0.00,0.00,0.00,0.0,1.353,1.820,1.046,11.405219
3,2036,"Cabbage, savoy, raw",Vegetables and Vegetable Products,2.00,0.10,6.10,0.00,2.27,3.1,0.013,0.007,0.049,3466.022131
4,5179,"Turkey, tom, wing, meat only, raw",Poultry Products,21.34,2.11,0.00,0.00,0.00,0.0,0.549,0.604,0.403,9.548843
5,4069,"Bread crumbs, dry, grated, plain",Baked Products,13.35,5.30,71.98,59.66,6.20,4.5,1.203,1.023,2.060,37.461742
6,3036,"Fish, pike, northern, native, raw",Finfish and Shellfish Products,19.50,0.80,0.00,0.00,0.00,0.0,0.100,0.100,0.200,3.308576
7,4437,"Grains, wheat, hard red winter","Cereals, Grains and Pasta",12.61,1.54,71.18,0.00,0.41,12.2,0.269,0.200,0.627,19.688678
8,565,"Chicken, broiler, meat only, raw",Poultry Products,21.39,3.08,0.00,0.00,0.00,0.0,0.790,0.900,0.750,3.095751
9,5934,"Spruce grouse, native, meat, raw",Poultry Products,24.00,1.00,0.00,0.00,0.00,0.0,0.200,0.100,0.000,2.066322


{'PROTValue': 104.72842869722693, 'FATValue': 8.606418151345277, 'CARBValue': 365.625, 'STARValue': 22.349675028402956, 'TSUGValue': 81.08205393862849, 'TDFValue': 119.97967008666623, 'TSATValue': 1.7928855965200896, 'MUFAValue': 1.7523700932214834, 'PUFAValue': 3.0207570100677583}
Meal Fitness:  1
